In [ ]:
# Courtney King 
# ML Semester Project 
# May 9 2022

In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import numpy as np
import random

In [2]:
# cleaned and processed data sets 
daily_numerical_features = pd.read_csv('daily_numerical_features.csv')
monthly_numerical_features = pd.read_csv('monthly_numerical_features.csv')
multinomial_numerical_features = pd.read_csv('multinomial_features.csv')
labels = pd.read_csv('labels.csv')

# had to use pickle because the df / csv did not properly preserve types 
with open('multinomial_dict.pkl', 'rb') as f:
    multinomial_dict = pickle.load(f)

In [3]:
labels['month'].value_counts()

5    68
4    36
Name: month, dtype: int64

## 1) Bayes with Multinomial Distribution

### Table 1a.  The imported dataset of weather types over time

In [4]:
pd.DataFrame.from_dict(multinomial_dict)

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,"[SN, SN, FG, FG, FG, FG, FG, FG, SN, SN, SN]","[BR, BR, SN, SN, BR]","[SN, SN, SN, BR, FG, BR, HZ, SN, BR, HZ, HZ, S...","[SN, SN, BR, HZ, BR, SN, BR, SN, BR, SN, BR, S...",[],[],[],[],"[HZ, SN, BR, SN, BR, SN, BR, HZ, BR, HZ, BR, H...","[SN, UP, SN, BR, UP, SN, BR, UP, SN, FZFG, BR,...",...,"[RA, BR, RA, RA, RA, RA, RA, SN, BR, RA, SN, H...","[RA, SN, RA, SN, SN, RA, RA, RA, SN, HZ, FZFG,...","[SN, BR, SN, SN, RA, RA, BR, RA, UP, RA, RA, S...","[BR, HZ, RA, RA, RA, SN, FZRA, SN, BR, HZ, RA,...","[SN, BR, SN, BR, SN, BR, UP, BR, SN, SN, BR, S...","[BR, BR, UP, FZRA, SN, BR, RA, FZRA, SN, FZFG,...","[RA, SN, FZFG, BR, UP, FG, BR, UP, HZ, HZ, BR,...","[SN, BR, UP, HZ, SN, HZ, HZ, SN, HZ, BR, HZ, B...","[HZ, SN, BR, UP, UP, BR, HZ, HZ, HZ, SN, SN, HZ]","[SN, UP, HZ]"
2,"[FG, FG, FG, FG, SN, FG, SN, SN, SN, SN, SN, SN]","[BR, SN, BR, BR, SN]","[SN, BR, HZ, RA, SN, BR, SN, BR, SN]","[SN, SN, HZ, SN, BR, HZ, SN, BR, HZ, SN, BR, H...",[],[],[],[],"[SN, BR, SN, BR, HZ, HZ, SN, BR, SN, SN, HZ, S...","[BR, HZ, BR, HZ, BR, UP, HZ, BR, HZ, BR, HZ, U...",...,"[RA, RA, RA, SN, BR, RA, BR, RA, SN, HZ, RA, S...","[UP, SN, BR, SN, BR, HZ, FZRA, SN, FZFG, BR, U...","[SN, BR, RA, HZ, RA, SN, RA, RA, RA, RA, HZ, R...","[RA, UP, RA, UP, SN, HZ, FZFG, BR, HZ, RA, FG,...","[FZFG, BR, UP, HZ, FG, FZFG, BR, HZ, FG, HZ, U...","[RA, RA, SN, BR, RA, BR, RA, SN, BR, UP, HZ, S...","[SN, BR, SN, BR, SN, BR, UP, RA, SN, HZ, BR, S...","[HZ, SN, HZ, SN, SN, FZFG, BR, HZ, FG, SN, BR,...","[SN, SN, HZ, SN, SN, SN, FG, BR, FZFG, BR, HZ,...",[]
3,"[SN, SN, SN, SN, SN, RA]","[SN, SN, SN]","[SN, SN, BR, SN, BR, SN, BR, SN, SN, SN]","[SN, BR, SN, BR, SN, BR, HZ, HZ, SN, SN, SN, H...",[],[],[],[],"[SN, BR, HZ, SN, BR, SN, FG, BR, HZ, SN, FG, B...","[SN, BR, UP, SN, UP, SN, UP, UP, UP, RA, UP, U...",...,"[SN, BR, RA, HZ, RA, BR, RA, SN, FZFG, BR, FG,...","[UP, SN, BR, HZ, SN, BR, HZ, SN, UP, UP, RA, U...","[SN, BR, RA, SN, BR, SN, BR, UP, SN, BR, UP, H...","[RA, SN, SN, SN, BR, HZ, HZ, UP, SN, HZ, RA, R...","[RA, RA, SN, SN, BR, UP, HZ, SN, BR, HZ, SN, B...","[HZ, SN, BR, UP, SN, FZFG, BR, FG, SN, BR, UP,...","[SN, BR, SN, BR, SN, BR, HZ, SN, BR, RA, SN, B...","[SN, SN, HZ, SN, HZ, HZ, HZ, SN, HZ, SN, HZ, S...","[SN, SN, SN, SN, HZ, SN, BR, HZ, SN, BR, SN, H...",[]
4,"[SN, SN, RA]","[SN, SN, SN, RA]","[SN, BR, RA, RA, SN, BR, SN, BR, SN]","[SN, BR, SN, BR, SN, FG, SN, RA, SN, BR]",[],[],[],[],"[SN, SN, SN, BR, RA, RA, RA]","[SN, UP, SN, BR, UP, SN, BR, UP, SN, BR, UP, U...",...,"[SN, BR, RA, SN, SN, BR, HZ, SN, BR, HZ, RA, S...","[RA, RA, RA, RA, SN, BR, HZ, RA, SN, BR, RA, R...","[SN, HZ, RA, SN, RA, SN, BR, HZ, UP, SN, SN, B...","[SN, BR, SN, BR, UP, RA, RA, RA, RA, SN, BR, H...","[RA, RA, HZ, RA, RA, SN, BR, RA, SN, BR, UP, H...","[SN, BR, SN, RA, RA, RA, RA, RA, RA, RA, RA, R...","[RA, RA, RA, RA, RA, RA, BR, HZ, RA, RA, SN, B...","[SN, BR, HZ, RA, SN, FG, BR, UP, RA, SN, UP, H...","[SN, UP, HZ, SN, BR, SN, FZFG, BR, FG, RA, SN,...",[]
5,"[RA, FG, RA, RA]","[RA, RA, RA, RA, RA, RA]","[RA, SN, BR, SN, BR, RA, RA, RA, RA, BR, RA, R...","[SN, BR, RA, RA, RA, FG, BR, RA, RA, RA, FG, B...",[],[],[],[],"[RA, BR, RA, BR, RA, SN, BR, RA, BR, RA, BR, R...","[RA, HZ, RA, BR, HZ, BR, BR]",...,"[SN, BR, HZ, RA, SN, RA, BR, SN, FZFG, BR, UP,...","[RA, RA, RA, RA, HZ, BR, RA, RA, RA, RA, RA, R...","[RA, BR, BR, RA, RA, RA]","[RA, RA, BR, RA, RA, RA, RA, RA, BR, RA, RA, R...","[RA, RA, RA, RA, RA, RA, RA, RA, RA, BR, RA, R...","[RA, SN, BR, UP, SN, BR, RA, RA, RA, BR, RA, S...","[RA, SN, BR, RA, RA, RA, RA, RA, RA, RA, RA, F...","[RA, RA, RA, RA]","[RA, RA, RA, RA, RA, RA, RA, RA, RA]",[]
6,"[RA, SN]",[RA],"[RA, RA, RA, RA, RA, RA, RA, FG, BR, RA, FG, B...","[RA, BR, RA, RA, RA, RA, RA, BR, RA, BR, RA, F...",[],[],[],[],"[RA, RA, BR, FG, BR, RA, BR, RA, RA, RA, FG, B...","[HZ, BR, FG, BR, HZ, BR, HZ, BR, BR, BR, HZ, H...",...,"[RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, HZ, R.

In [5]:
# Optimize
# Annual dictionary for NUM_MONTHS before the break 
NUM_MONTHS = 3 # try starting with Jan -March only

# for each year, combine month's value a single list of values 
annual_vals = dict()
for year in multinomial_dict.items():
    years_values = []
    for month in year[1].items(): # iterate year values (month, values)
        years_values+=month[1] # the month values; 
        if(month[0] >= NUM_MONTHS): 
            annual_vals[year[0]] = years_values 
            break # only want the months less than April 

In [6]:
# Make a frequency dictionary of these values over each year
# step 1) for each year, for each type, set to zero 
freq_dict = dict()
existing_weather_types = ['PL', 'DU', 'DZ', 'RA', 'BLSN', 'SN', 'TS', 'FZFG', 'FZRA', 'FG', 'UP', 'BR', 'HZ']
for year in annual_vals.keys():
    freq_dict[year] = dict()
    for t in existing_weather_types: 
        freq_dict[year][t] = 0
        
# step 2) count values in dictionary created above 
for year, vals in annual_vals.items():
    for v in vals:
        freq_dict[year][v]+=1

freq_df = pd.DataFrame.from_dict(freq_dict).transpose();

# Attach class labels to freq_df 
# since they are from different data sets
temp = []
for x in freq_dict.keys():    
    temp.append(int( labels[labels['Year'] == x].month))
    if x == 2020: break
temp.append('?') # 2021, 2022 are not present in the data set.
temp.append('?')

freq_df['class_labels'] = temp
freq_df = freq_df.drop([2000,2001,2002,2003]).drop(['DU', 'TS'], axis=1)
# dropped empty years/features in the data 
freq_df = freq_df[:-2]

### Table 1a. Data as Frequency Table of weather types 

In [7]:
# now the data is in a usable format the class label has been joined to each record
freq_df

,PL,DZ,RA,BLSN,SN,FZFG,FZRA,FG,UP,BR,HZ,class_labels
1996,0,0,1,0,17,0,0,11,0,0,0,5
1997,0,0,0,0,7,0,0,0,0,6,0,4
1998,0,0,1,0,16,0,0,1,0,9,6,4
1999,0,0,0,0,31,0,0,0,0,31,24,4
2004,0,0,0,0,31,0,0,4,0,25,29,4
2005,0,0,4,0,27,6,1,8,69,26,13,4
2006,0,0,2,0,52,5,0,5,37,25,16,5
2007,2,2,47,0,49,4,0,5,1,30,23,4
2008,0,0,46,0,63,0,0,1,5,24,15,5
2009,0,0,27,1,53,3,0,5,7,40,26,5


In [8]:
# getting indexing headaches  

def Counts( f=None, c=None):
    table=class_freq.copy()
    if f == None:
        if c == None: return float(table[:].values.sum()) # count sample space
        else: return float(table.loc[:,c].values.sum())   # count c_j in space 
    if c == None: return float(table.loc[f,:].values.sum()) # count f_i in space 
    return float(table.loc[f,c])  # count f_i & c_j in space


In [9]:
# Reserve some years for the test set before training

years = freq_df.index
perc = 0.2 # use percentage of years for the test set
num_test= int ( np.ceil (len(years)*perc) ) 

# test and train will be an index mappings to the data 
# remove some years at random 
test = [] # get a random year in the data range
for i in random.sample(range(0, len(years)), num_test): test.append(years[i])

train =[y for y in years if y not in test]

# to remove the test records from the data set
getTest = lambda x : trainingData[trainingData.index==test[x]] 

trainingData = freq_df
testData = pd.concat([getTest(i) for i in range(len(test))])
trainingData = trainingData.drop(test)

# data reserved for later testing 
# remaining data to use by classifer 
print( len(testData),len(trainingData) )

5 16


In [10]:
# Create Likelihoods Table

# count feature, class combinations 
class_freq = pd.DataFrame(trainingData[trainingData['class_labels']==4].sum())
class_freq['5'] = trainingData[trainingData['class_labels']==5].sum()
class_freq = class_freq.drop('class_labels')
class_freq.columns = ['4', '5']

# compute probabilities for each feature, given each class
# P( for f in F | c ) for c in C

In [11]:
# Optimization 

# Laplace Smoothing
# there are some features which have no datapoint 
# use alpha smoothing to avoid having a zero likelihoods for new features being seen 
alpha = 0.01 # smoothing param
k = len(class_freq) # number of features ro use in denominator

# start with the count of c,j found together in the data
# then divide them by each class summation
likelihoods = class_freq.copy()

likelihoods+=alpha # add alpha to each feature
likelihoods['4'] /= (Counts(c='4')+(k*alpha)) # accounting for alpha in numerator
likelihoods['5'] /= (Counts(c='5')+(k*alpha))
print(likelihoods['4'].sum(), likelihoods['5'].sum() )

1.0 1.0


### Table 1b. Likelihoods Table P(F|C)

In [12]:
likelihoods

,4,5
PL,0.003210,0.000021
DZ,0.001609,0.000021
RA,0.214561,0.156043
BLSN,0.000008,0.002073
SN,0.269000,0.379812
FZFG,0.024826,0.016444
FZRA,0.004811,0.000021
FG,0.032831,0.045185
UP,0.057649,0.100614
BR,0.206555,0.182731


In [13]:
testData 

,PL,DZ,RA,BLSN,SN,FZFG,FZRA,FG,UP,BR,HZ,class_labels
2005,0,0,4,0,27,6,1,8,69,26,13,4
2017,0,0,30,0,40,11,0,11,15,30,23,5
2013,0,0,52,0,41,3,1,4,9,32,18,5
2011,0,0,44,1,34,3,1,3,20,28,7,5
2018,0,0,34,0,39,8,2,8,12,41,16,5


In [14]:
t = Counts(c='4') + Counts(c='5') # total 
y = [ (Counts(c='4') / t) , (Counts(c='5') / t ) ] # P(C)
l0 = likelihoods['4'] # P(x|C1)
l1 = likelihoods['5'] # P(x|C2)

pred = ['?']*len(testData) # the past 2 years are not yet known
true = [ str(x) for x in testData['class_labels'] ] 
for i in range(len(testData)):
    if testData.iloc[i,-1] == '?': 
        continue
    t = testData.iloc[i,:-1] # testData point i
    test0 = np.product([t*l for t,l in zip(t, l0) if t!=0])*y[0] #P(C1|x)
    test1 = np.product([t*l for t,l in zip(t, l1) if t!=0])*y[1] #P(C1|x)
    pred[i] = '4' if test0 > test1 else '5'

print(true)
print(pred)
accuracy = sum( [ pred[x]==true[x] for x in range(len(testData)) ] ) / len(testData)
print(accuracy)

['4', '5', '5', '5', '5']
['4', '4', '4', '4', '4']
0.2





## Classifier 2) Bayes with Gaussian Distribution

### Table 2a. The initial numerical features 

In [15]:
monthly_numerical_features = pd.read_csv('monthly_numerical_features.csv')
# Optimize 

# Shouldn't use the year after the melt point, start with only Jan-Mar range for each year 
monthly_numerical_features = monthly_numerical_features[monthly_numerical_features['x_month'] <= 3]
monthly_numerical_features = monthly_numerical_features.reset_index(drop=True)


In [16]:
monthly_numerical_features[monthly_numerical_features['Year']==1999]
                          

,max_temp,mean_temp,min_temp,station_pressure,x_month,Year,class_month
2,-0.4,-12.6,-24.9,29.21,2,1999,4
3,19.8,7.0,-5.8,29.24,3,1999,4
4,0.4,-11.9,-24.2,29.43,1,1999,4


In [17]:
# reserve some years for the test set 

years = monthly_numerical_features['Year'].unique()
perc = 0.2 # use percentage of years for the test set
num_test= int ( np.ceil (len(years)*perc) ) 

# test and train will be an index mappings to the data 
# remove some years at random 
test = [] # get a random year in the data range
for i in random.sample(range(0, len(years)), num_test): test.append(years[i])

train =[y for y in years if y not in test]

trainingData = monthly_numerical_features

testData = pd.DataFrame(columns=trainingData.columns)
for i in test: 
    testData = pd.concat([ testData, monthly_numerical_features[monthly_numerical_features['Year']==i]])
trainingData = trainingData.drop(list(testData.index))

trainingData.reset_index(drop=True)
testData.reset_index(drop=True)
print(len(trainingData), len(testData))

50 14


In [18]:
trainingData

,max_temp,mean_temp,min_temp,station_pressure,x_month,Year,class_month
0,33.2,21.3,9.3,29.33,3,1998,4
1,14.0,3.5,-7.1,29.04,2,1998,4
2,-0.4,-12.6,-24.9,29.21,2,1999,4
3,19.8,7.0,-5.8,29.24,3,1999,4
4,0.4,-11.9,-24.2,29.43,1,1999,4
5,4.8,-3.3,-11.3,29.33,1,2000,5
6,18.1,6.0,-6.2,29.27,2,2000,5
7,28.3,16.3,4.2,29.19,3,2000,5
8,20.6,10.3,0.0,29.47,2,2001,5
9,19.2,8.2,-2.9,29.37,3,2001,5


In [19]:

feats = ['station_pressure', 'mean_temp'] 
cls = [4,5]

# only features 
f_params = []
for f in feats :
    f_params.append( [trainingData[f].mean() , trainingData[f].std()] )
    
# reduce by class 
cf_params = []
for c in cls:
    for f in feats:
        cf_params.append( [trainingData[ trainingData['class_month']==c ][f].mean() , 
             trainingData[ trainingData['class_month']==c ][f].std() ])

In [20]:
cf_params

[[29.4232, 0.17976187953326864],
 [4.268000000000001, 11.042860438612212],
 [29.465999999999998, 0.17774044746952403],
 [4.372000000000001, 7.382215114719971]]

In [21]:
f_params

[[29.444600000000005, 0.17823637137697976],
 [4.320000000000001, 9.296411420872166]]

In [22]:
# f & c (mean , std) , 4 & station pressure 
# (  )
cf_params

[[29.4232, 0.17976187953326864],
 [4.268000000000001, 11.042860438612212],
 [29.465999999999998, 0.17774044746952403],
 [4.372000000000001, 7.382215114719971]]

In [23]:
# Predict

# given the  f ( sig , mu ) for this x, 
# what is the label? 
X = []
for i in zip( testData[feats[0]] , testData[feats[1]]): 
    X.append(i)
X 

[(29.2, 13.5),
 (29.16, 11.5),
 (29.29, 17.6),
 (29.28, -24.0),
 (29.49, 13.0),
 (29.58, 5.7),
 (29.28, 10.0),
 (29.6, -0.5),
 (29.45, 14.4),
 (29.47, -21.2),
 (29.24, 1.2),
 (29.23, 11.8),
 (29.6, -3.0),
 (29.59, -7.1)]

In [24]:
prior = len(trainingData['class_month'] ==4) / (len(trainingData['class_month'] ==5)+len(trainingData['class_month'] ==4))


In [25]:
# Helper for labelBayes 
def Gauss(x, mean, std):
    exponent = math.exp(-((x - mean) ** 2 / (2 * std ** 2)))
    return (1 /(math.sqrt(2 * math.pi) * std)) * exponent

In [26]:
X[0]

(29.2, 13.5)

## Classifier 3 Bayes With Hidden Markov 

### Table 3a.Data Table showing sequential weather patterns  

In [27]:
# go through imported Multinomial dictionary to 
# form the most recent observation chain for each year
temp_col, temp_val=[], [] 
for year, x in annual_vals.items():
    # the chains must contain 20 observations
    if(len(x))>=20:
        temp_col.append(year)
        temp_val.append(x[-20:])
        
chains = pd.DataFrame( {temp_col[x] : temp_val[x] for x in range(len(temp_col))} )
chains

,1996,1998,1999,2004,2005,2006,2007,2008,2009,2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,SN,HZ,SN,SN,UP,UP,RA,RA,SN,SN,...,SN,SN,UP,UP,BR,SN,BR,BR,UP,SN
1,SN,SN,BR,FG,SN,SN,RA,BR,RA,BR,...,UP,BR,RA,SN,HZ,BR,HZ,SN,UP,FZFG
2,FG,BR,SN,BR,UP,UP,SN,SN,RA,RA,...,SN,HZ,RA,BR,HZ,HZ,SN,BR,SN,BR
3,FG,HZ,BR,HZ,SN,SN,RA,RA,SN,SN,...,SN,SN,RA,UP,UP,RA,FZFG,RA,FZFG,FG
4,FG,RA,SN,SN,UP,UP,SN,SN,SN,RA,...,HZ,BR,RA,HZ,SN,RA,BR,RA,BR,SN
5,FG,SN,BR,FG,UP,UP,BR,SN,UP,RA,...,UP,SN,RA,RA,HZ,SN,RA,RA,UP,FZFG
6,SN,BR,SN,BR,UP,SN,HZ,SN,RA,RA,...,UP,SN,UP,SN,RA,SN,FG,RA,HZ,BR
7,FG,SN,BR,HZ,UP,UP,RA,BR,SN,UP,...,RA,UP,RA,UP,RA,SN,SN,BR,FG,HZ
8,SN,BR,SN,SN,UP,HZ,SN,RA,UP,RA,...,SN,HZ,SN,HZ,RA,SN,BR,RA,SN,FG
9,SN,SN,BR,HZ,SN,SN,BR,SN,HZ,RA,...,SN,SN,SN,HZ,RA,RA,RA,SN,BR,SN


In [28]:
# get all unique observation types existing in chains 
obs= set()
for x in chains:
    for y in chains[x].values:
        obs.add(y)
obs

{'BR', 'FG', 'FZFG', 'FZRA', 'HZ', 'PL', 'RA', 'SN', 'UP'}

### Table 3b. Transitional Probabilities to find param: A_{i,<=j}

In [29]:
trans_dict = dict.fromkeys(obs)
for o in obs: trans_dict[o] = dict.fromkeys(obs, 0)
    
# columns are years, records 
for c in chains.columns:
    # compare each weather state with the proceeding
    for i in range(len(chains[c])-1): 
        # increment transition count in dictionary 
        trans_dict[chains[c][i]][chains[c][i+1]]+=1
        
trans_freq_df = pd.DataFrame.from_dict(trans_dict)
# column -> row : freq 
trans_freq_df

,FZRA,RA,PL,FG,UP,HZ,BR,FZFG,SN
FZRA,0,1,0,0,1,0,1,0,0
RA,0,48,1,1,13,9,14,0,13
PL,0,1,0,0,0,1,0,0,0
FG,0,2,0,3,1,2,2,0,6
UP,1,7,0,1,11,2,14,0,13
HZ,0,1,0,0,9,4,16,0,6
BR,0,7,0,3,0,0,0,7,48
FZFG,0,0,0,0,0,1,0,0,6
SN,1,29,0,7,16,17,16,0,36


In [30]:
obs_totals = [sum(trans_freq_df[c]) for c in trans_freq_df.columns]
obs_totals

[2, 96, 1, 15, 51, 36, 63, 7, 128]

### Table 3c (as probabilities)

In [31]:
# transition from column -> row probabilities
# trans_prob_df['HZ']['RA'] = 0.25 ( 'HZ'-> 'RA' )
trans_prob_df = pd.DataFrame(trans_freq_df)
for i in range(len(trans_prob_df)):
    trans_prob_df.iloc[:,i] /= obs_totals[i] 
trans_prob_df

,FZRA,RA,PL,FG,UP,HZ,BR,FZFG,SN
FZRA,0.0,0.010417,0.0,0.000000,0.019608,0.000000,0.015873,0.0,0.000000
RA,0.0,0.500000,1.0,0.066667,0.254902,0.250000,0.222222,0.0,0.101562
PL,0.0,0.010417,0.0,0.000000,0.000000,0.027778,0.000000,0.0,0.000000
FG,0.0,0.020833,0.0,0.200000,0.019608,0.055556,0.031746,0.0,0.046875
UP,0.5,0.072917,0.0,0.066667,0.215686,0.055556,0.222222,0.0,0.101562
HZ,0.0,0.010417,0.0,0.000000,0.176471,0.111111,0.253968,0.0,0.046875
BR,0.0,0.072917,0.0,0.200000,0.000000,0.000000,0.000000,1.0,0.375000
FZFG,0.0,0.000000,0.0,0.000000,0.000000,0.027778,0.000000,0.0,0.046875
SN,0.5,0.302083,0.0,0.466667,0.313725,0.472222,0.253968,0.0,0.281250


### Table 3d) Initial Probabilities Table for HMM (PI param)

In [32]:
# Pi, the intiial probabilities of each state
PI = pd.DataFrame({ trans_prob_df.columns[x] : 
                   [obs_totals[x]/ (sum(obs_totals))] for x in range(len(trans_prob_df.columns)) })
PI

,FZRA,RA,PL,FG,UP,HZ,BR,FZFG,SN
0,0.005013,0.240602,0.002506,0.037594,0.12782,0.090226,0.157895,0.017544,0.320802
